In [1]:
import os
import pickle
import pandas as pd
from collections import defaultdict

In [ ]:
#symptomatic, location

In [2]:
questions=["age","tipo","tecnica","family","history","symtomatic",
           "prosthesis","birads","density_mammo","calcifications_benign","density_echo","ganglio_mamo","lymph_benign","lymph_suspicious","parenchymal_distortion","simple_cyst","ductal_ectasia",
          "nodules_echo_num","nodules_echo_description","nodules_echo_shape","nodules_echo_margin","nodules_echo_echogenicity","nodules_echo_location","nodules_echo_size","nodules_echo_known","nodules_echo_stable"]
questions=["tipo","tecnica","family",
           "prosthesis","birads","density_mammo","calcifications_benign","ganglio_mamo","density_echo","lymph_benign","lymph_suspicious","simple_cyst","ductal_ectasia","nodules_echo_num","nodules_echo_shape","nodules_echo_margin","nodules_echo_echogenicity","nodules_echo_known","nodules_echo_stable"]


In [3]:
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report
def calculate_f1(pred,truth,point=False,average='macro'):
    pred_clean={}
    truth_clean={}
    for key,real in truth.items():
        if key in pred:
            
            predicted=pred[key] 
            if point==True:
                predicted=predicted.split(".")[0]+"."
            elif point=="mm":
                if "unknown" in predicted:
                    predicted=predicted.split(".")[0]+"."
                else:
                    predicted=predicted.split("mm")[0]+"mm."
                
            pred_clean[key]=predicted.lower()
            truth_clean[key]=str(real).lower()
    # Get ground truth and predicted labels
    y_true = list(truth_clean.values())
    y_pred = list(pred_clean.values())
    
    # Define the valid labels based on the truth only
    valid_labels = sorted(set(y_true))
    
    # Replace out-of-scope predictions with something invalid
    # e.g., keep them but let them be counted as wrong
    y_pred_cleaned = [
        p if p in valid_labels else 'INVALID' for p in y_pred
    ]
    
    # Now align with true values
    y_true_final = []
    y_pred_final = []
    
    for yt, yp in zip(y_true, y_pred_cleaned):
        y_true_final.append(yt)
        y_pred_final.append(yp)
    
    # Compute F1 over valid labels only
    f1 = f1_score(
        y_true_final,
        y_pred_final,
        labels=valid_labels,   # Only ground-truth classes
        average='macro'        # or whatever average you want
    )
    report = classification_report(
        y_true,
        y_pred_cleaned,
        labels=valid_labels,     # Only ground truth classes
        zero_division=0,         # Avoid warnings for undefined precision/recall
        digits=4                 # Optional: better precision
    )
    print(report)
    return f1


In [4]:
def calculate_acc(pred,truth,point=False):
    acc=0
    total=0
    for key,real in truth.items():
        if key in pred:
            total+=1
            predicted=pred[key]    
            if point==True:

                predicted=predicted.split(".")[0]+"."

            elif point=="mm":
                if "unknown" in predicted:
                    predicted=predicted.split(".")[0]+"."
                else:
                    predicted=predicted.split("mm")[0]+"mm."
            if predicted.lower()==str(real).lower():
                acc+=1
            # else:
            #     print(key,real,predicted)
        # else:
        #     print(key)
    return acc/total

In [5]:

accuracies=defaultdict(list)
f1s=defaultdict(list)
models=["biogpt"+"51e535e65"+"no_copy"+"description"+"16batch"+"final_answer_freeze","biogptQA2e55e66no_copydescription8batchfinal","biogptQA1e55e66no_copydescription8batchfinal","clinicalt5"+"1e47"+"no_copy"+"description"+"16batch"+"final"+"tokenized","clinicalt5"+"1e45"+"no_copy"+"description"+"16batch"+"final"+"tokenized","clinicalt55e58no_copydescription16batchfinal","clinicalt57e510no_copydescription16batchfinal","clinicalt5"+"7e57"+"no_copy"+"description"+"16batch"+"final","clinicalt5"+"7e57"+"no_copy"+"description"+"16batch"+"final"+"tokenized","clinicalt5"+"7e53"+"no_copy"+"description"+"16batch"+"final"+"tokenized","biogpt1e57no_copydescription16batchfinal","biogpt21e51e57no_copydescription16batchfinal","biogpt21e51e55no_copydescription16batchfinal","biogpt"+"1e57"+"no_copy"+"description"+"16batch"+"final"+"no_info"]
for model in models:
    accuracies["model"].append(model)
    f1s["model"].append(model)
    for tipo in questions:
        print(tipo,model)
        
        with open(f"Generativos/truth_dic/{tipo}.pkl", "rb") as file:
                truth=pickle.load(file)
        with open(f"Generativos/results_dic_{tipo}/{model}.pkl", "rb") as file:
                output=pickle.load(file)
        if tipo=="age":
            print(truth)
            print(output)
        if tipo!="nodules_echo_size":
            acc=calculate_acc(output,truth,point=True)
            f1=calculate_f1(output,truth,point=True)
        else:
            acc=calculate_acc(output,truth,point="mm")
            
            f1=calculate_f1(output,truth,point="mm")
        accuracies[tipo].append(acc)
        f1s[tipo].append(f1)



tipo biogpt51e535e65no_copydescription16batchfinal_answer_freeze
                                    precision    recall  f1-score   support

                    biopsy report.     1.0000    0.7500    0.8571        20
  nodal staging ultrasound report.     1.0000    0.5000    0.6667         6
normal control or revision report.     0.9585    0.9946    0.9763       186

                         micro avg     0.9621    0.9575    0.9598       212
                         macro avg     0.9862    0.7482    0.8334       212
                      weighted avg     0.9636    0.9575    0.9563       212

tecnica biogpt51e535e65no_copydescription16batchfinal_answer_freeze
                             precision    recall  f1-score   support

mammography and ultrasound.     0.9737    0.9867    0.9801       150
    only mammography study.     0.0000    0.0000    0.0000         2
     only ultrasound study.     0.9831    0.9667    0.9748        60

                  micro avg     0.9763    0.9717    0.

In [6]:
data=pd.DataFrame(accuracies)
data=data.set_index("model")
data['average'] = data.mean(axis=1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
data

,tipo,tecnica,family,prosthesis,birads,density_mammo,calcifications_benign,ganglio_mamo,density_echo,lymph_benign,lymph_suspicious,simple_cyst,ductal_ectasia,nodules_echo_num,nodules_echo_shape,nodules_echo_margin,nodules_echo_echogenicity,nodules_echo_known,nodules_echo_stable,average
model,,,,,,,,,,,,,,,,,,,,
biogpt51e535e65no_copydescription16batchfinal_answer_freeze,0.957547,0.971698,0.956989,0.989247,0.967742,0.967742,0.983871,0.967742,0.940217,0.940860,0.940860,0.956989,0.978495,0.908602,0.890244,0.682927,0.841463,0.682927,0.813953,0.912638
biogptQA2e55e66no_copydescription8batchfinal,0.962264,0.985849,0.983871,0.989247,0.978495,0.989247,0.983871,0.967742,0.940217,0.967742,0.983871,0.973118,1.000000,0.951613,0.926829,0.878049,0.926829,0.817073,0.906977,0.953311
biogptQA1e55e66no_copydescription8batchfinal,0.962264,0.985849,0.983871,0.989247,0.978495,0.989247,0.983871,0.967742,0.940217,0.967742,0.983871,0.973118,1.000000,0.951613,0.926829,0.878049,0.926829,0.817073,0.906977,0.953311
clinicalt51e47no_copydescription16batchfinaltokenized,0.952830,0.981132,0.951613,0.983871,0.763441,0.978495,0.973118,0.887097,0.777174,0.854839,0.876344,0.962366,0.946237,0.860215,0.914634,0.853659,0.926829,0.865854,0.790698,0.900023
clinicalt51e45no_copydescription16batchfinaltokenized,0.952830,0.948113,0.876344,0.994624,0.741935,0.876344,0.919355,0.951613,0.728261,0.919355,0.946237,0.951613,0.973118,0.844086,0.841463,0.804878,0.865854,0.743902,0.744186,0.874953
clinicalt55e58no_copydescription16batchfinal,0.948113,0.943396,0.946237,0.983871,0.607527,0.865591,0.913978,0.908602,0.869565,0.892473,0.892473,0.946237,0.973118,0.768817,0.853659,0.865854,0.926829,0.621951,0.674419,0.863301
clinicalt57e510no_copydescription16batchfinal,0.943396,0.933962,0.881720,0.978495,0.736559,0.854839,0.849462,0.946237,0.820652,0.892473,0.919355,0.946237,0.973118,0.725806,0.865854,0.853659,0.914634,0.634146,0.465116,0.849248
clinicalt57e57no_copydescription16batchfinal,0.773585,0.830189,0.784946,0.887097,0.596774,0.704301,0.811828,0.817204,0.603261,0.790323,0.779570,0.806452,0.849462,0.682796,0.743902,0.768293,0.634146,0.597561,0.604651,0.740334
clinicalt57e57no_copydescription16batchfinaltokenized,0.952830,0.971698,0.930108,0.994624,0.758065,0.962366,0.973118,0.935484,0.793478,0.913978,0.951613,0.973118,0.983871,0.903226,0.902439,0.841463,0.914634,0.780488,0.790698,0.906700


In [7]:
biogpt_acc = list(data.loc["biogpt1e57no_copydescription16batchfinal",:])[:-1]

In [8]:
data=pd.DataFrame(f1s)
data=data.set_index("model")
data['average'] = data.mean(axis=1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
data

,tipo,tecnica,family,prosthesis,birads,density_mammo,calcifications_benign,ganglio_mamo,density_echo,lymph_benign,lymph_suspicious,simple_cyst,ductal_ectasia,nodules_echo_num,nodules_echo_shape,nodules_echo_margin,nodules_echo_echogenicity,nodules_echo_known,nodules_echo_stable,average
model,,,,,,,,,,,,,,,,,,,,
biogpt51e535e65no_copydescription16batchfinal_answer_freeze,0.833354,0.651641,0.900559,0.947159,0.928337,0.971086,0.983792,0.777334,0.906203,0.910299,0.756968,0.958949,0.935259,0.908388,0.729654,0.310275,0.354248,0.682639,0.462963,0.784690
biogptQA2e55e66no_copydescription8batchfinal,0.813757,0.660590,0.939738,0.947159,0.914840,0.989566,0.983792,0.741667,0.902118,0.943385,0.943903,0.972942,1.000000,0.950988,0.763600,0.468318,0.578044,0.817046,0.538012,0.835235
biogptQA1e55e66no_copydescription8batchfinal,0.813757,0.660590,0.939738,0.947159,0.914840,0.989566,0.983792,0.741667,0.902118,0.943385,0.943903,0.972942,1.000000,0.950988,0.763600,0.468318,0.578044,0.817046,0.538012,0.835235
clinicalt51e47no_copydescription16batchfinaltokenized,0.620904,0.656696,0.886409,0.924298,0.721501,0.978594,0.973024,0.580676,0.796299,0.800611,0.714832,0.962181,0.859940,0.859419,0.701422,0.360923,0.576106,0.865674,0.294372,0.743888
clinicalt51e45no_copydescription16batchfinaltokenized,0.705560,0.642961,0.759847,0.972268,0.731361,0.869031,0.921246,0.487603,0.687037,0.869449,0.848843,0.951443,0.926002,0.841665,0.587740,0.472504,0.485815,0.739209,0.288288,0.725678
clinicalt55e58no_copydescription16batchfinal,0.609474,0.625483,0.882379,0.907539,0.577872,0.866412,0.915711,0.476056,0.843205,0.815805,0.692460,0.945608,0.912023,0.753049,0.589631,0.492844,0.577714,0.589669,0.268519,0.702182
clinicalt57e510no_copydescription16batchfinal,0.597426,0.627460,0.841864,0.869382,0.703584,0.845781,0.845442,0.486188,0.754061,0.806452,0.719513,0.945474,0.906504,0.697387,0.613178,0.359511,0.572355,0.605769,0.222222,0.685240
clinicalt57e57no_copydescription16batchfinal,0.627207,0.581720,0.673830,0.860531,0.490496,0.621582,0.855487,0.451039,0.491017,0.745358,0.511690,0.852242,0.803032,0.716808,0.458915,0.462519,0.401754,0.621078,0.319192,0.607658
clinicalt57e57no_copydescription16batchfinaltokenized,0.620904,0.648788,0.842077,0.972268,0.826945,0.963253,0.973055,0.483333,0.778049,0.870225,0.858075,0.972986,0.947214,0.902311,0.661339,0.485128,0.507789,0.777174,0.294372,0.757120


In [9]:
biogpt_f1 = list(data.loc["biogpt1e57no_copydescription16batchfinal",:])[:-1]

In [10]:
TIPO=["biopsy report", "nodal staging ultrasound report", "normal control or revision report"]
TECNICA=["only ultrasound study", "only mammography study", "mammography and ultrasound"]
FAMILY=["no family history", "first degree", "second degree"]
PROSTHESIS=["no prosthesis","yes prosthesis"]
BIRADS=["BI-RADS 0","BI-RADS 1","BI-RADS 2","BI-RADS 3","BI-RADS 4A","BI-RADS 4B","BI-RADS 4C","BI-RADS 5","BI-RADS 6"]
DENSITY_MAMMO=["ACR A","ACR B","ACR C","ACR D","unknown density mammo"]
CALCIFICATIONS_BENIGN=["no calcifications","yes calcifications"]
GANGLIO_MAMO=["no ganglio","yes ganglio"]
DENSITY_ECHO=["homogeneous fibroglandular","heterogeneous fibroglandular","fibroglandular and fat","homogeneous fatty","unknown density echo"]
LYMPH_BENIGN=["no lymph benign","yes lymph benign"]
LYMPH_SUSPICIOUS=["no lymph suspicious","yes lymph suspicious"]
SIMPLE_CYST=["no cyst","yes cyst"]
DUCTAL_ECTASIA=["no ectasia","yes ectasia"]
NODULES_ECHO=["no nodules", "yes nodules"]
NODULES_SHAPE=["oval","round","lobulated","irregular","unknown shape"]
NODULES_MARGIN=["circumscribed","spiculated","indistinct","not circumscribed","unknown margin"]
NODULES_ECHOGENICITY=["hypoechoic", "isoechoic", "heterogeneous","complex cystic and solid","unknown echogenicity"]
NODULES_KNOWN=["no known", "yes known"]
NODULES_STABLE=["grown stable","shrunk stable", "yes stable"]

DICTIONARY={"tipo":TIPO,"tecnica":TECNICA,"family":FAMILY,"prosthesis":PROSTHESIS,"birads":BIRADS,"density_mammo":DENSITY_MAMMO,"calcifications_benign":CALCIFICATIONS_BENIGN,
            "ganglio_mamo":GANGLIO_MAMO,"density_echo":DENSITY_ECHO,"lymph_benign":LYMPH_BENIGN,"lymph_suspicious":LYMPH_SUSPICIOUS,"simple_cyst":SIMPLE_CYST,"ductal_ectasia":DUCTAL_ECTASIA,
           "nodules_echo": NODULES_ECHO,"nodules_shape":NODULES_SHAPE,"nodules_margin":NODULES_MARGIN, "nodules_echogenicity":NODULES_ECHOGENICITY, "nodules_known":NODULES_KNOWN, "nodules_stable":NODULES_STABLE}

def evaluate_per_question(predicted, tested, DICTIONARY, average="macro"):
    """
    Evaluate model predictions per question type.
    
    Parameters:
    - predicted: array of predicted label indices (flattened from all folds)
    - tested: array of true label indices (same shape as predicted)
    - DICTIONARY: dict mapping each question to its list of class names
    """
    
    # Step 1: Build global index → (question, class_name) mapping
    idx_to_question_value = {}
    offset = 0
    question_offsets = {}
    for question, class_list in DICTIONARY.items():
        question_offsets[question] = offset
        for i, label in enumerate(class_list):
            idx_to_question_value[offset + i] = (question, label)
        offset += len(class_list)

    # Step 2: Group predictions by question
    per_question_true = defaultdict(list)
    per_question_pred = defaultdict(list)

    for true_idx, pred_idx in zip(tested, predicted):
        q_true, _ = idx_to_question_value[true_idx]
        # You can check if q_true == q_pred here for safety if needed
        per_question_true[q_true].append(true_idx)
        per_question_pred[q_true].append(pred_idx)

    # Step 3: Classification reports

    f1s={}
    accuracies={}
    for question, true_labels in per_question_true.items():
        pred_labels = per_question_pred[question]
        label_names = DICTIONARY[question]
        start = question_offsets[question]
        end = start + len(label_names)
        question_label_ids = list(range(start, end))
        f1=f1_score(true_labels,pred_labels,labels=sorted(set(true_labels)),average=average)
        f1s[question]=f1
        accuracy=accuracy_score(true_labels,pred_labels)
        accuracies[question]=accuracy
    return f1s,accuracies        


In [11]:
import re
import numpy as np
models=["predicted_bio","predicted_biomed__no_prior","predicted_biomed_8epoch","predicted_biomed","predicted_biomed_8epoch_truncate","predicted_blue2","predicted_biomed_8epoch_no_info"]
accuracies=defaultdict(list)
f1s=defaultdict(list)
for model in models:
    model_truth=re.sub("predicted","tested",model)
    truth=np.load(f"BERT/{model_truth}.npy")
    outputs=np.load(f"BERT/{model}.npy")
    f1s_model,accuracies_model=evaluate_per_question(outputs,truth,DICTIONARY)
    accuracies["model"].append(model)
    f1s["model"].append(model)
    for question in DICTIONARY.keys():
        accuracies[question].append(accuracies_model[question])
        f1s[question].append(f1s_model[question])
        

In [12]:
data=pd.DataFrame(f1s)
data=data.set_index("model")
data['average'] = data.mean(axis=1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
data

,tipo,tecnica,family,prosthesis,birads,density_mammo,calcifications_benign,ganglio_mamo,density_echo,lymph_benign,lymph_suspicious,simple_cyst,ductal_ectasia,nodules_echo,nodules_shape,nodules_margin,nodules_echogenicity,nodules_known,nodules_stable,average
model,,,,,,,,,,,,,,,,,,,,
predicted_bio,0.705560,0.660590,0.914149,0.972268,0.666319,0.936159,0.983792,0.880437,0.824742,0.907903,0.900161,0.983765,0.982405,0.918314,0.532364,0.430252,0.369209,0.852871,0.402930,0.780221
predicted_biomed__no_prior,0.789279,0.656631,0.979385,0.972268,0.682715,0.977972,0.978405,0.925778,0.852800,0.905641,0.943903,0.978372,1.000000,0.945474,0.536792,0.343912,0.379573,0.816828,0.303797,0.787870
predicted_biomed_8epoch,0.856571,0.652865,0.979385,0.972268,0.668367,0.953006,0.967607,0.827778,0.892884,0.903226,0.884136,0.978372,1.000000,0.945608,0.537783,0.355781,0.582956,0.841440,0.386364,0.799284
predicted_biomed,0.856571,0.656758,0.894035,0.972268,0.625527,0.978620,0.978372,0.880437,0.914371,0.892028,0.922757,0.972942,1.000000,0.934392,0.533775,0.352737,0.375051,0.829167,0.299145,0.782576
predicted_biomed_8epoch_truncate,0.901885,0.648933,0.925355,0.972268,0.748627,0.987289,0.983792,0.800729,0.913214,0.933118,0.922757,0.972942,1.000000,0.929205,0.554001,0.446470,0.511540,0.828861,0.402930,0.809680
predicted_blue2,0.697039,0.626707,0.635569,0.941620,0.235385,0.530751,0.897491,0.490411,0.602857,0.889220,0.806809,0.929519,0.963743,0.861751,0.356486,0.340209,0.311615,0.705742,0.538012,0.650576
predicted_biomed_8epoch_no_info,0.867633,0.652763,0.961797,0.972268,0.787957,0.972246,0.983792,0.880437,0.908455,0.926322,0.927485,0.978372,1.000000,0.929022,0.546015,0.507332,0.582956,0.753754,0.299145,0.812513


In [13]:
biomed_f1 = list(data.loc["predicted_biomed",:])[:-1]

In [14]:
data=pd.DataFrame(accuracies)
data=data.set_index("model")
data['average'] = data.mean(axis=1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
data

,tipo,tecnica,family,prosthesis,birads,density_mammo,calcifications_benign,ganglio_mamo,density_echo,lymph_benign,lymph_suspicious,simple_cyst,ductal_ectasia,nodules_echo,nodules_shape,nodules_margin,nodules_echogenicity,nodules_known,nodules_stable,average
model,,,,,,,,,,,,,,,,,,,,
predicted_bio,0.952830,0.985849,0.983871,0.994624,0.951613,0.973118,0.983871,0.983871,0.940860,0.946237,0.973118,0.983871,0.994624,0.919355,0.890244,0.841463,0.890244,0.853659,0.860465,0.942305
predicted_biomed__no_prior,0.962264,0.981132,0.989247,0.994624,0.940860,0.973118,0.978495,0.989247,0.913978,0.946237,0.983871,0.978495,1.000000,0.946237,0.890244,0.804878,0.914634,0.817073,0.837209,0.939044
predicted_biomed_8epoch,0.971698,0.976415,0.989247,0.994624,0.956989,0.973118,0.967742,0.978495,0.930108,0.946237,0.967742,0.978495,1.000000,0.946237,0.890244,0.841463,0.939024,0.841463,0.837209,0.943503
predicted_biomed,0.971698,0.981132,0.973118,0.994624,0.940860,0.978495,0.978495,0.983871,0.935484,0.935484,0.978495,0.973118,1.000000,0.935484,0.890244,0.829268,0.914634,0.829268,0.813953,0.938828
predicted_biomed_8epoch_truncate,0.976415,0.971698,0.978495,0.994624,0.956989,0.983871,0.983871,0.973118,0.946237,0.962366,0.978495,0.973118,1.000000,0.930108,0.914634,0.865854,0.926829,0.829268,0.860465,0.947708
predicted_blue2,0.952830,0.943396,0.924731,0.989247,0.688172,0.768817,0.897849,0.962366,0.752688,0.930108,0.940860,0.930108,0.989247,0.865591,0.646341,0.804878,0.756098,0.707317,0.906977,0.860927
predicted_biomed_8epoch_no_info,0.971698,0.976415,0.989247,0.994624,0.967742,0.967742,0.983871,0.983871,0.940860,0.956989,0.978495,0.978495,1.000000,0.930108,0.890244,0.841463,0.939024,0.756098,0.813953,0.940049


In [15]:
biomed_acc = list(data.loc["predicted_biomed",:])[:-1]

In [16]:
from scipy.stats import wilcoxon


# Perform Wilcoxon signed-rank test (one-sided: model1 > model2)
statistic, p_value = wilcoxon(biogpt_acc, biomed_acc, alternative='greater')

print(f"Wilcoxon statistic: {statistic}")
print(f"p-value: {p_value}")

# Perform Wilcoxon signed-rank test (one-sided: model1 > model2)
statistic, p_value = wilcoxon(biogpt_f1, biomed_f1, alternative='greater')

print(f"Wilcoxon statistic f1: {statistic}")
print(f"p-value: {p_value}")

Wilcoxon statistic: 83.0
p-value: 0.09523009656757148
Wilcoxon statistic f1: 106.0
p-value: 0.08128587957814465


In [19]:
BioMedBERT= [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.4909, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.251, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.1429, 0.1688, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.1404, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.124, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.2197, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8106, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8983, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.2077, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.116, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5846, 1.0, 1.0, 1.0, 1.0, 1.0, 0.7448, 1.0, 1.0, 1.0, 1.0, 0.7993, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5135, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.1061, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0921, 1.0, 1.0, 0.791, 1.0, 0.1715, 0.8349, 0.0428, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0492, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.2575, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.4213, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.215, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.2746, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.1416, 1.0, 1.0, 1.0, 0.9432, 1.0, 1.0, 1.0, 1.0, 0.1577, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5547, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [17]:
biogpt=[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.2917,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.4909,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.316,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0527,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.251,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.6876,
 0.146,
 1.0,
 0.1382,
 0.8059,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.8059,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.2644,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.1616,
 1.0,
 1.0,
 1.0,
 0.2175,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.1423,
 1.0,
 1.0,
 1.0,
 1.0,
 0.5211,
 1.0,
 0.5211,
 1.0,
 1.0,
 0.5211,
 1.0,
 1.0,
 0.5211,
 1.0,
 1.0,
 1.0,
 1.0,
 0.5211,
 1.0,
 0.8087,
 0.1618,
 1.0,
 1.0,
 1.0,
 1.0,
 0.8589,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.1786,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.3294,
 1.0,
 1.0,
 0.1156,
 1.0,
 1.0,
 1.0,
 0.961,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.1717,
 1.0,
 1.0,
 1.0,
 0.1757,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.1212,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9459,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.1167,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.6714,
 0.1913,
 0.1913,
 1.0,
 1.0,
 0.433,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.788,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.8255,
 0.5399,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.2685,
 1.0,
 1.0,
 1.0,
 0.8731,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.6953,
 1.0,
 1.0,
 1.0,
 0.8013,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.5633,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.7586,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.7297,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.901,
 1.0,
 0.2157,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.4595,
 1.0,
 0.3007,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.2157,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9432,
 1.0,
 1.0,
 1.0,
 1.0,
 0.4039,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.5547,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.115,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0]

In [20]:
from scipy.stats import wilcoxon


# Perform Wilcoxon signed-rank test (one-sided: model1 > model2)
statistic, p_value = wilcoxon(biogpt, BioMedBERT, alternative='greater')

print(f"Wilcoxon statistic: {statistic}")
print(f"p-value: {p_value}")

Wilcoxon statistic: 617.0
p-value: 0.9858794618422767
